# Imports


In [38]:
# -*- coding: utf-8 -*-

from bs4 import BeautifulSoup as soupify
from os import path
from requests import get
from importlib import reload
import pandas as pd
import numpy as np

import re
import unicodedata
import wrangle
import prepare

In [ ]:
# Functions

Define a function named basic_clean. It should take in a string and apply some basic text cleaning to it:

Lowercase everything
Normalize unicode characters
Replace anything that is not a letter, number, whitespace or a single quote.


In [11]:
original = wrangle.get_search_csv()

In [21]:
def basic_clean(text):
    """ 
    Purpose:
        
    ---
    Parameters:
        
    ---
    Returns:

    """
    text = text.lower()

    text = unicodedata.normalize('NFKD', text)\
                        .encode('ascii', 'ignore')\
                        .decode('utf-8', 'ignore')
    
    text = re.sub(r"[^a-z0-9'\s]", '', text)
    return text

Define a function named tokenize. It should take in a string and tokenize all the words in the string.



In [14]:
from nltk.tokenize.toktok import ToktokTokenizer

def tokenize(text):
    """ 
    Purpose:
        takes a string and tokenizes all words in t
    ---
    Parameters:
        
    ---
    Returns:
    
    """ 
    tokenizer = ToktokTokenizer()

    text = tokenizer.tokenize(text, return_str=True)

    return text 

Define a function named stem. It should accept some text and return the text after applying stemming to all the words.



In [15]:
from nltk import PorterStemmer

def stem(text):
    """ 
    Purpose:
        to apply stemming to input text
    ---
    Parameters:
        text: the text to be stemmed
    ---
    Returns:
        text: text that has had stemming applied to it
    """

    #create the nltk stemmer object
    ps = PorterStemmer()    

    stems = [ps.stem(word) for word in text.split()]
    text = ' '.join(stems)

    return text

Define a function named lemmatize. It should accept some text and return the text after applying lemmatization to each word.



In [16]:
from nltk import WordNetLemmatizer
def lemmatize(text):
    """ 
    Purpose:
        applies lemmatization to input text 
    ---
    Parameters:
        text: the text to be lemmatized
    ---
    Returns:
        text: text that has been lemmatized
    """
    #create lemmatize object
    wnl = WordNetLemmatizer()

    lemmas = [wnl.lemmatize(word) for word in text.split()]
    text = ' '.join(lemmas)

    return text



Define a function named remove_stopwords. It should accept some text and return the text after removing all the stopwords.

This function should define two optional parameters, extra_words and exclude_words. These parameters should define any additional stop words to include, and any words that we don't want to remove.



In [17]:
from nltk.corpus import stopwords

def remove_stopwords(text, extra_words=None, exclude_words=None):
    """ 
    Purpose:
        to remove stopwords from input text 
    ---
    Parameters:
        text: text from which to remove stop words
    ---
    Returns:
        text: text that has had stopwords removed
    """

    stopwords_list = stopwords.words('english')

    if extra_words != None:
        stopwords_list.extend(extra_words)

    if exclude_words != None:
        for w in exclude_words:
            stopwords_list.remove(w)

    words = text.split()

    filtered_words = [w for w in words if w not in stopwords_list]

    print()
    print('Removed {} stopwords'.format(len(words) - len(filtered_words)))
    print('---')

    text = ' '.join(filtered_words)

    return text


In [55]:
def clean(text, extra_words=None, exclude_words=None):
    """ 
    Purpose:
        performs basic clean, tokenization, and removal of stopwords on input text
    ---
    Parameters:
        text
        extra_words
        exclude_words
    ---
    Returns:
        text
    """

    text = basic_clean(text)
    
    text = tokenize(text)

    text = remove_stopwords(text, extra_words, exclude_words)

    return text

## Use your data from the acquire to produce a dataframe of the READMEs.

* Produce the following columns:

        title to hold the title
        language to hold the language
        readme.txt to hold the original article/post content
        clean to hold the normalized and tokenized original with the stopwords removed.
        stemmed to hold the stemmed version of the cleaned data.
        lemmatized to hold the lemmatized version of the cleaned data.


In [19]:
df = wrangle.get_search_csv()

In [36]:
df = df.dropna()

In [40]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [41]:
df['is_english'] = df.readme_txt.apply(isEnglish)

In [50]:
#columns containing non-english characters
df = df.drop([4, 13, 66, 86])

In [44]:
df[df.is_english == False ]

,Unnamed: 0,repo,language,readme_txt,is_english
0,0,homerchen19/nba-go,JavaScript,"<p align=center>\n<img src=""https://user-image...",False
1,1,hegaojian/JetpackMvvm,Kotlin,[![Platform][1]][2] [![GitHub license][3]][4] ...,False
3,3,chonyy/AI-basketball-analysis,Python,"<p align=center>\n <img src=""./static/img/a...",False
4,4,hegaojian/WanAndroid,Kotlin,[![Platform][1]][2] [![GitHub license][3]][4] ...,False
5,5,bttmly/nba,JavaScript,# nba\n*Node.js client for nba.com API endpoin...,False
6,6,smuyyh/SprintNBA,Java,# :basketball: SprintNBA\n\n## 完整的NBA第三方Androi...,False
10,10,kshvmdn/nba.js,JavaScript,## nba.js\n\n[![npm version](https://badge.fur...,False
13,13,hegaojian/MvvmHelper,Kotlin,#### :chicken::chicken::chicken: 这是一个快速开发的框架，使...,False
15,15,jrbadiabo/Bet-on-Sibyl,Jupyter Notebook,======================![Sibyl Logo](Images/Sib...,False
20,20,xwjdsh/nba-live,Go,```text\n __ ___ ...,False


In [54]:
# from nltk.classify.textcat import TextCat
#loop for guessing language of input text
# for i in df.index:

#     txtcat = TextCat()
#     guess = txtcat.guess_language(df.iloc[i]['readme_txt'])
#     print(i + 19, guess)

In [51]:
def prep_text(df):
    """ 
    Purpose:
        
    ---
    Parameters:
        
    ---
    Returns:
    
    """

    df['clean'] = df.readme_txt.apply(clean)
    df['stemmed'] = df.clean.apply(stem)
    df['lemmatized'] = df.clean.apply(lemmatize)

    return df

In [52]:
df = prep_text(df)

Removed 201 stopwords
---
Removed 49 stopwords
---
Removed 2 stopwords
---
Removed 218 stopwords
---
Removed 401 stopwords
---
Removed 0 stopwords
---
Removed 50 stopwords
---
Removed 8 stopwords
---
Removed 158 stopwords
---
Removed 21 stopwords
---
Removed 44 stopwords
---
Removed 110 stopwords
---
Removed 500 stopwords
---
Removed 494 stopwords
---
Removed 31 stopwords
---
Removed 427 stopwords
---
Removed 76 stopwords
---
Removed 1 stopwords
---
Removed 175 stopwords
---
Removed 731 stopwords
---
Removed 68 stopwords
---
Removed 58 stopwords
---
Removed 358 stopwords
---
Removed 52 stopwords
---
Removed 15 stopwords
---
Removed 48 stopwords
---
Removed 100 stopwords
---
Removed 116 stopwords
---
Removed 0 stopwords
---
Removed 134 stopwords
---
Removed 320 stopwords
---
Removed 27 stopwords
---
Removed 74 stopwords
---
Removed 88 stopwords
---
Removed 8 stopwords
---
Removed 272 stopwords
---
Removed 123 stopwords
---
Removed 389 stopwords
---
Removed 81 stopwords
---
Removed 187 s

In [53]:
df.head(20)

,Unnamed: 0,repo,language,readme_txt,is_english,clean,stemmed,lemmatized
0,0,homerchen19/nba-go,JavaScript,"<p align=center>\n<img src=""https://user-image...",False,p aligncenter img srchttpsuserimagesgithubuser...,p aligncent img srchttpsuserimagesgithubuserco...,p aligncenter img srchttpsuserimagesgithubuser...
1,1,hegaojian/JetpackMvvm,Kotlin,[![Platform][1]][2] [![GitHub license][3]][4] ...,False,platform12 github license34 github license56 1...,platform12 github license34 github license56 1...,platform12 github license34 github license56 1...
2,2,naoyashiga/Dunk,Swift,# Dunk\n![](https://raw.githubusercontent.com/...,True,dunk httpsrawgithubusercontentcomnaoyashigadun...,dunk httpsrawgithubusercontentcomnaoyashigadun...,dunk httpsrawgithubusercontentcomnaoyashigadun...
3,3,chonyy/AI-basketball-analysis,Python,"<p align=center>\n <img src=""./static/img/a...",False,p aligncenter img srcstaticimganalysisgif widt...,p aligncent img srcstaticimganalysisgif width9...,p aligncenter img srcstaticimganalysisgif widt...
5,5,bttmly/nba,JavaScript,# nba\n*Node.js client for nba.com API endpoin...,False,nba nodejs client nbacom api endpoints npm ins...,nba nodej client nbacom api endpoint npm insta...,nba nodejs client nbacom api endpoint npm inst...
6,6,smuyyh/SprintNBA,Java,# :basketball: SprintNBA\n\n## 完整的NBA第三方Androi...,False,basketball sprintnba nbaandroidnba apinbanbanb...,basketbal sprintnba nbaandroidnba apinbanbanba...,basketball sprintnba nbaandroidnba apinbanbanb...
7,7,linouk23/NBA-Player-Movements,Python,# NBA Player Movements\n\nThis is a script for...,True,nba player movements script visualization nba ...,nba player movement script visual nba game raw...,nba player movement script visualization nba g...
8,8,FaridSafi/react-native-basketball,JavaScript,# react-native-basketball\n\nReact-Native clon...,True,reactnativebasketball reactnative clone facebo...,reactnativebasketbal reactn clone facebook bas...,reactnativebasketball reactnative clone facebo...
9,9,TryKickoff/kickoff,CSS,![Kickoff](http://i.imgur.com/bfMlVwe.jpg)\n\n...,True,kickoffhttpiimgurcombfmlvwejpg lightweight fro...,kickoffhttpiimgurcombfmlvwejpg lightweight fro...,kickoffhttpiimgurcombfmlvwejpg lightweight fro...
10,10,kshvmdn/nba.js,JavaScript,## nba.js\n\n[![npm version](https://badge.fur...,False,nbajs npm versionhttpsbadgefuryiojsnbajssvghtt...,nbaj npm versionhttpsbadgefuryiojsnbajssvghttp...,nbajs npm versionhttpsbadgefuryiojsnbajssvghtt...


In [ ]:
reload(acquire)

<module 'acquire' from '/Users/sinao/codeup-data-science/natural-language-processing-exercises/acquire.py'>